#**Project:** Restaurant Order Analysis(Python + SQL)
#**Created by:** [Samir Alikperov](https://www.linkedin.com/in/samiralikperov/)
---------------------------
## **Source:** Maven Analytics
## **Link:** [Project](https://app.mavenanalytics.io/guided-projects/d7167b45-6317-49c9-b2bb-42e2a9e9c0bc)


In [ ]:
# This command installs the mysql-connector-python library, which allows Python to interact with a MySQL database.
!pip install mysql-connector-python

In [ ]:
import mysql.connector

# Function to connect to the MySQL database
def connect_to_mysql():
    try:
        connection = mysql.connector.connect(
            host='host',    # Database server address
            user='user',         # Username for connection
            password='password', # Password for the user
            database='database' # Name of the database
        )
        if connection.is_connected():
            print("Successful connection to the MySQL database")
        return connection
    except mysql.connector.Error as e:
        print(f"Connection error: {e}")
        return None

# Connecting to the database
connection = connect_to_mysql()


Successful connection to the MySQL database


In [ ]:
# Function to execute SQL queries from a file
def execute_sql_script(connection, file_path):
    try:
        # Create a cursor object to interact with the database
        cursor = connection.cursor()

        # Open the SQL file and read its contents
        with open(file_path, 'r') as file:
            sql_script = file.read()

        # Split the script into individual commands using ';' as a delimiter
        sql_commands = sql_script.split(';')

        # Execute each command in the list
        for command in sql_commands:
            if command.strip():  # Skip any empty commands
                cursor.execute(command)  # Execute the current command
                connection.commit()  # Commit the transaction to the database
                print(f"Query executed: {command.strip()[:100]}...")  # Print a preview of the executed query

        # Close the cursor after all commands are executed
        cursor.close()
        print("All queries executed successfully.")
    except mysql.connector.Error as e:
        # Handle any errors related to SQL execution
        print(f"Error executing SQL query: {e}")
    except FileNotFoundError:
        # Handle the case where the specified file is not found
        print(f"File {file_path} not found.")

# Example of executing an SQL script
sql_file_path = r'C:\Users\path\create_restaurant_db.sql'  # Specify the path to the SQL file
execute_sql_script(connection, sql_file_path)



Query executed: DROP SCHEMA IF EXISTS restaurant_db...
Query executed: CREATE SCHEMA restaurant_db...
Query executed: USE restaurant_db...
Query executed: --
-- Table structure for table `order_details`
--

CREATE TABLE order_details (
  order_details_id ...
Query executed: --
-- Table structure for table `menu_items`
--

CREATE TABLE menu_items (
  menu_item_id SMALLINT N...
Query executed: --
-- Insert data into table order_details
--

INSERT INTO order_details VALUES (1, 1, '2023-01-01',...
Query executed: --
-- Insert data into table menu_items
--

INSERT INTO menu_items VALUES (101, 'Hamburger', 'Americ...
All queries executed successfully.


In [ ]:
import pandas as pd

# Function to execute a SELECT SQL query and return the result as a DataFrame
def execute_select_query(connection, query):
    if connection.is_connected():
        cursor = connection.cursor()  # Create a cursor to interact with the database
        cursor.execute(query)  # Execute the given SQL query

        result = cursor.fetchall()  # Fetch all rows from the executed query
        columns = [i[0] for i in cursor.description]  # Get column names from the cursor description

        # Create a DataFrame using the query results and column names
        df = pd.DataFrame(result, columns=columns)
        cursor.close()  # Close the cursor

        return df  # Return the resulting DataFrame
    else:
        # If the connection is not established, print a message
        print("Database connection is not established.")
        return None

# Example of a simple SELECT query
query = "SELECT * FROM menu_items LIMIT 10"
df = execute_select_query(connection, query)

# Display the DataFrame if the query was successful
if df is not None:
    display(df)


,menu_item_id,item_name,category,price
0,101,Hamburger,American,12.95
1,102,Cheeseburger,American,13.95
2,103,Hot Dog,American,9.00
3,104,Veggie Burger,American,10.50
4,105,Mac & Cheese,American,7.00
5,106,French Fries,American,7.00
6,107,Orange Chicken,Asian,16.50
7,108,Tofu Pad Thai,Asian,14.50
8,109,Korean Beef Bowl,Asian,17.95
9,110,Pork Ramen,Asian,17.95


# **Objective 1**  
*Explore the items table*  
*Your first objective is to better understand the items table by finding the number of rows in the table, the least and most expensive items, and the item prices within each category.*



#### View the menu_items table and write a query to find the number of items on the menu


In [ ]:
query_menu_tabl = """ SELECT *
                      FROM menu_items """
menu_tabl = execute_select_query(connection, query_menu_tabl)
menu_tabl

,menu_item_id,item_name,category,price
0,101,Hamburger,American,12.95
1,102,Cheeseburger,American,13.95
2,103,Hot Dog,American,9.00
3,104,Veggie Burger,American,10.50
4,105,Mac & Cheese,American,7.00
5,106,French Fries,American,7.00
6,107,Orange Chicken,Asian,16.50
7,108,Tofu Pad Thai,Asian,14.50
8,109,Korean Beef Bowl,Asian,17.95
9,110,Pork Ramen,Asian,17.95


In [ ]:
query_menu_cnt = """SELECT COUNT(*) as count_items
                    FROM menu_items"""
menu_cnt = execute_select_query(connection, query_menu_cnt)
menu_cnt

,count_items
0,32


#### What are the least and most expensive items on the menu?

In [ ]:
query_menu_price = """SELECT price as expensive_price,
                             (SELECT price
                              FROM menu_items
                              ORDER BY price
                              LIMIT 1) as least_price
                      FROM menu_items
                      ORDER BY price DESC
                      LIMIT 1
                      """
menu_price = execute_select_query(connection, query_menu_price)
menu_price

,expensive_price,least_price
0,19.95,5.00


#### How many Italian dishes are on the menu?

In [ ]:
query_menu_itl = """SELECT COUNT(*) as itl_cnt
                    FROM menu_items
                    WHERE category = 'Italian'
                    """
menu_itl_cnt = execute_select_query(connection, query_menu_itl)
menu_itl_cnt

,itl_cnt
0,9


#### What are the least and most expensive Italian dishes on the menu?

In [ ]:
query_itl_price = """SELECT price as expensive_itl_price,
                             (SELECT price
                              FROM menu_items
                              WHERE category = 'Italian'
                              ORDER BY price
                              LIMIT 1) as least_itl_price
                      FROM menu_items
                      WHERE category = 'Italian'
                      ORDER BY price DESC
                      LIMIT 1
                      """
itl_price = execute_select_query(connection, query_itl_price)
itl_price

,expensive_itl_price,least_itl_price
0,19.95,14.50


#### How many dishes are in each category?

In [ ]:
all_dish_by_cat = """SELECT category,
                            COUNT(*) as numbers_item
                     FROM menu_items
                     GROUP BY category
                     """
dish_category = execute_select_query(connection, all_dish_by_cat)
dish_category

,category,numbers_item
0,American,6
1,Asian,8
2,Mexican,9
3,Italian,9


#### What is the average dish price within each category?

In [ ]:
avg_price_by_category = """SELECT category,
                                  ROUND(AVG(price),2) as avg_price
                           FROM menu_items
                           GROUP BY category
                          """
dish_category = execute_select_query(connection, avg_price_by_category)
dish_category

,category,avg_price
0,American,10.07
1,Asian,13.48
2,Mexican,11.80
3,Italian,16.75


In [ ]:
avg_price_by_category = """SELECT category,
                                  ROUND(AVG(price),2) as avg_price
                           FROM menu_items
                           GROUP BY category
                          """
dish_category = execute_select_query(connection, avg_price_by_category)
dish_category

,category,avg_price
0,American,10.07
1,Asian,13.48
2,Mexican,11.80
3,Italian,16.75


## Objective 2  
**Explore the orders table**  
*Your second objective is to better understand the orders table by finding the date range, the number of items within each order, and the orders with the highest number of items.*


#### View the order_details table. What is the date range of the table?

In [ ]:
query_order_tabl = """ SELECT *
                      FROM order_details """
order_tabl = execute_select_query(connection, query_order_tabl)
order_tabl

,order_details_id,order_id,order_date,order_time,item_id
0,1,1,2023-01-01,0 days 11:38:36,109.0
1,2,2,2023-01-01,0 days 11:57:40,108.0
2,3,2,2023-01-01,0 days 11:57:40,124.0
3,4,2,2023-01-01,0 days 11:57:40,117.0
4,5,2,2023-01-01,0 days 11:57:40,129.0
...,...,...,...,...,...
12229,12230,5369,2023-03-31,0 days 22:05:04,109.0
12230,12231,5369,2023-03-31,0 days 22:05:04,129.0
12231,12232,5369,2023-03-31,0 days 22:05:04,120.0
12232,12233,5369,2023-03-31,0 days 22:05:04,122.0


#### How many orders were made within this date range?

In [ ]:
query_order_date_range = """ SELECT MIN(order_date) as min_date,
                                    MAX(order_date) as max_date
                             FROM order_details """
order_date_range = execute_select_query(connection, query_order_date_range)
order_date_range

,min_date,max_date
0,2023-01-01,2023-03-31


In [ ]:
query_order_cnt = """ SELECT COUNT(DISTINCT order_id) as num_orders
                      FROM order_details
                      """
order_cnt = execute_select_query(connection, query_order_cnt)
order_cnt

,num_orders
0,5370


#### How many items were ordered within this date range?

In [ ]:
query_order_cnt_items = """ SELECT COUNT(*) as num_items
                            FROM order_details
                            """
order_items = execute_select_query(connection, query_order_cnt_items)
order_items

,num_items
0,12234


#### Which orders had the most number of items?

In [ ]:
query_order_most_items = """ SELECT order_id, COUNT(item_id) as num_items
                             FROM order_details
                             GROUP BY order_id
                             ORDER BY num_items DESC, order_id
                             LIMIT 20

                            """
order_most_items = execute_select_query(connection, query_order_most_items)
order_most_items

,order_id,num_items
0,330,14
1,440,14
2,443,14
3,1957,14
4,2675,14
5,3473,14
6,4305,14
7,1274,13
8,1569,13
9,1685,13


#### How many orders had more than 12 items?

In [ ]:
query_order_more_twelve = """ with twelve AS
                                (SELECT order_id, COUNT(item_id) as num_items
                                FROM order_details
                                GROUP BY order_id
                                HAVING num_items > 12
                                ORDER BY num_items DESC, order_id)

                                SELECT COUNT(*) as cnt_orders
                                FROM twelve


                            """
order_more_twelve = execute_select_query(connection, query_order_more_twelve)
order_more_twelve

,cnt_orders
0,20


## Objective 3
**Analyze customer behavior**
*Your final objective is to combine the items and orders tables, find the least and most ordered categories, and dive into the details of the highest spend orders.*


#### Combine the menu_items and order_details tables into a single table

In [ ]:
query_combine_tabl = """ SELECT *
                         FROM order_details o
                         LEFT JOIN menu_items m
                             ON m.menu_item_id = o.item_id


                          """
combine_tabl = execute_select_query(connection, query_combine_tabl)
combine_tabl

,order_details_id,order_id,order_date,order_time,item_id,menu_item_id,item_name,category,price
0,1,1,2023-01-01,0 days 11:38:36,109.0,109.0,Korean Beef Bowl,Asian,17.95
1,2,2,2023-01-01,0 days 11:57:40,108.0,108.0,Tofu Pad Thai,Asian,14.50
2,3,2,2023-01-01,0 days 11:57:40,124.0,124.0,Spaghetti,Italian,14.50
3,4,2,2023-01-01,0 days 11:57:40,117.0,117.0,Chicken Burrito,Mexican,12.95
4,5,2,2023-01-01,0 days 11:57:40,129.0,129.0,Mushroom Ravioli,Italian,15.50
...,...,...,...,...,...,...,...,...,...
12229,12230,5369,2023-03-31,0 days 22:05:04,109.0,109.0,Korean Beef Bowl,Asian,17.95
12230,12231,5369,2023-03-31,0 days 22:05:04,129.0,129.0,Mushroom Ravioli,Italian,15.50
12231,12232,5369,2023-03-31,0 days 22:05:04,120.0,120.0,Steak Torta,Mexican,13.95
12232,12233,5369,2023-03-31,0 days 22:05:04,122.0,122.0,Chips & Salsa,Mexican,7.00


#### What were the least and most ordered items? What categories were they in?

In [ ]:
query_ordered_items = """WITH tabl AS (
    SELECT item_name, category, COUNT(order_details_id) AS num_purchases
    FROM order_details o
    LEFT JOIN menu_items m ON m.menu_item_id = o.item_id
    GROUP BY item_name, category
),
min_max AS (
    SELECT MIN(num_purchases) AS min_purchases,
           MAX(num_purchases) AS max_purchases
    FROM tabl
)

SELECT item_name,
       category,
       num_purchases
FROM tabl
WHERE num_purchases = (SELECT min_purchases FROM min_max)
   OR num_purchases = (SELECT max_purchases FROM min_max);

                          """
combine_ordered_items = execute_select_query(connection, query_ordered_items)
combine_ordered_items

,item_name,category,num_purchases
0,Hamburger,American,622
1,Chicken Tacos,Mexican,123



#### What were the top 5 orders that spent the most money?

In [ ]:
query_most_many = """
    SELECT order_id, SUM(price) AS total_sum_price
    FROM order_details o
    LEFT JOIN menu_items m ON m.menu_item_id = o.item_id
    GROUP BY order_id
    ORDER BY total_sum_price DESC
    LIMIT 5

                          """
combine_most_many = execute_select_query(connection, query_most_many)
combine_most_many

,order_id,total_sum_price
0,440,192.15
1,2075,191.05
2,1957,190.10
3,330,189.70
4,2675,185.10



#### View the details of the highest spend order. Which specific items were purchased?

In [ ]:
query_details_spend_order = """
   -- Step 1: Retrieve the top spending orders
WITH top_orders AS (
    SELECT order_id, SUM(price) AS total_sum_price
    FROM order_details o
    LEFT JOIN menu_items m ON m.menu_item_id = o.item_id
    GROUP BY order_id
    ORDER BY total_sum_price DESC
    LIMIT 5
)

-- Step 2: Retrieve details of those orders, including item names
SELECT o.order_id,
       m.category,
       SUM(price) AS item_total_price,
       COUNT(item_id) AS item_total_num
FROM order_details o
LEFT JOIN menu_items m ON m.menu_item_id = o.item_id
WHERE o.order_id IN (SELECT order_id FROM top_orders)
GROUP BY o.order_id, m.category
ORDER BY o.order_id, item_total_price DESC;

                          """
combine_details_spend_order = execute_select_query(connection, query_details_spend_order)
combine_details_spend_order

,order_id,category,item_total_price,item_total_num
0,330,Asian,87.40,6
1,330,Italian,50.40,3
2,330,Mexican,42.90,4
3,330,American,9.00,1
4,440,Italian,132.25,8
5,440,Asian,22.95,2
6,440,Mexican,20.95,2
7,440,American,16.00,2
8,1957,Italian,84.30,5
9,1957,Mexican,42.85,3


In [ ]:
# Install the dtale package, which is a tool for visualizing and exploring pandas dataframes
!pip install dtale

# Import the dtale library
import dtale

# Launch a D-Tale session to explore the 'combine_tabl' dataframe interactively
dtale.show(combine_tabl)